# Testing Database Design Quality
## Assignment 2: Relational Model for Uber’s Ride-Sharing System 
### Aryan Dutt, Prince Raj, Maria Camila Villamizar

Function to analyze normalization

Main Execution

In [154]:
import pandas as pd
from itertools import combinations

def load_data(file_path):
    """Loads data from a CSV file."""
    df = pd.read_csv(file_path)
    return df.dropna(how="all")

def extract_df():
    """Extracts the DataFrame from the address.csv table."""
    file_path = "database/new/address.csv"
    return load_data(file_path)

def find_fds(df):
    """Finds functional dependencies by iterating over columns and assigning each one as RHS."""
    fds = []
    columns = df.columns.tolist()
    partition_cache = {}  # Cache for partition refinement

    for rhs in columns:
        if rhs == "address_id":  # address_id should not be RHS
            continue

        lhs_candidates = [col for col in columns if col != rhs]
        minimal_lhs = explore_lhs(df, lhs_candidates, rhs, partition_cache)
        
        for lhs in minimal_lhs:
            fds.append((lhs, rhs))

    # Apply post-processing to remove unnecessary FDs if address_id is primary key
    fds = filter_unnecessary_fds(fds, df)
    return fds

def explore_lhs(df, lhs_candidates, rhs, partition_cache):
    """Explores combinations of the left-hand side (LHS) using partition refinement."""
    minimal_lhs = []
    
    for r in range(1, len(lhs_candidates) + 1):
        for lhs in combinations(lhs_candidates, r):
            lhs_list = list(lhs)
            if check_fd(df, lhs_list, rhs, partition_cache):
                if not any(set(lhs_list).issuperset(set(existing)) for existing in minimal_lhs):
                    minimal_lhs.append(lhs_list)
    
    return minimal_lhs

def check_fd(df, lhs, rhs, partition_cache):
    """Checks if a functional dependency exists between LHS and RHS using partition intersection."""
    lhs_key = tuple(lhs)
    
    if lhs_key in partition_cache:
        lhs_partition = partition_cache[lhs_key]
    else:
        lhs_partition = compute_partition(df, lhs)
        partition_cache[lhs_key] = lhs_partition

    # Compute intersection of partitions (key concept in DFD)
    lhs_rhs_partition = compute_partition(df, lhs + [rhs])
    
    return len(lhs_partition) == len(lhs_rhs_partition)  # Valid FD if partitions have the same size

def filter_unnecessary_fds(fds, df):
    """Removes unnecessary FDs if address_id determines all columns."""
    primary_key = "address_id"
    
    # Check if address_id is a primary key (which we already confirmed)
    if df[primary_key].nunique() == len(df):
        return [(["address_id"], rhs) for rhs in df.columns if rhs != "address_id"]
    
    return fds  # If address_id is not a primary key, return original results

def compute_partition(df, attributes):
    """Computes the partition of a set of attributes."""
    partition = {}
    for index, row in df.iterrows():
        key = tuple(row[attr] for attr in attributes)
        if key not in partition:
            partition[key] = set()
        partition[key].add(index)
    return partition

def prune_candidates(lhs_sets):
    """Efficient pruning using a dictionary of subsets."""
    pruned = {}
    for lhs in lhs_sets:
        key = frozenset(lhs)
        if not any(existing.issubset(key) for existing in pruned):
            pruned[key] = lhs
    return list(pruned.values())

def manage_memory(partition_cache, max_size=1000):
    """Removes old partitions to free memory if the cache exceeds a certain size."""
    if len(partition_cache) > max_size:
        keys_to_remove = list(partition_cache.keys())[:len(partition_cache) - max_size]
        for key in keys_to_remove:
            del partition_cache[key]

# Run the algorithm
df = extract_df()
fds = find_fds(df)

# Display results
for lhs, rhs in fds:
    print(f"{', '.join(lhs)} → {rhs}")


address_id → city_id
address_id → postal_code
address_id → description
address_id → latitude
address_id → longitude
address_id → street_num
address_id → street_name
